In [13]:
from bs4 import BeautifulSoup
from requests import get
from collections import defaultdict
from sets import Set
import csv

dirtyDupedUnpairedCourseNameList = []  # list of only courseNames with dupes and no pairs
dirtyUnpairedCourseNameSet = []  # list of only courseNames with no dupes but no pairs
dirtyUnpairedCourseNameHash = {}  # boolean hash of courseNames 
scrapeCleanedUnpairedRows = []  # all rows of department without pairs
UnpairedRowsWithSeparateChainingDict = defaultdict(list)  # hash to pair the above
cleanedDepartmentalCourseList = []  # list of tuples with guaranteed summer and non-summer pairs

url = 'https://cape.ucsd.edu/responses/Results.aspx?Name=&CourseNumber=cse'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
response = get(url, headers=headers)
html_soup = BeautifulSoup(response.text, 'html.parser')
table = html_soup.find('table')
table_body = table.find('tbody')
rows = table_body.find_all('tr')

 
def setLink(link):
    resetAllLists()
    global url 
    url = link
    global response 
    response = get(url, headers=headers)
    global html_soup 
    html_soup = BeautifulSoup(response.text, 'html.parser')
    global table 
    table = html_soup.find('table')
    global table_body
    table_body = table.find('tbody')
    global rows 
    rows = table_body.find_all('tr')


def convertHtmlToString(tag):
    readable = tag.text.strip().encode("utf-8") 
    if readable[-1] == '%':
        return readable[0:len(readable) - 2]
    return readable


def scrapeNameList(dirtyList):
    for row in rows:
        rowArray = row.find_all('td')
        if (convertHtmlToString(rowArray[2])[0:2] == 'S1' or convertHtmlToString(rowArray[2])[0:2] == 'S2' or convertHtmlToString(rowArray[2])[0:2] == 'S3' or convertHtmlToString(rowArray[2])[0:2] == 'SU') and convertHtmlToString(rowArray[1])[0:2] != 'No' :
            year = row.find_all('td')[1].text.strip().encode("utf-8")
            year = year.replace(" ", "")
            year = year[0:len(year) - 3]
            dirtyList.append(year)  # Get rid of empty values\



def createSetList(duped):
    unduped = list(Set(duped))
    unduped.sort()
    return unduped


def booleanHashCourseName(courseNameSet):
    for i in courseNameSet:
        dirtyUnpairedCourseNameHash[i] = 1

        
def scrapeCapeRows():
    for row in rows:
        rowList = row.find_all('td')
        courseName = rowList[1].text.strip().encode("utf-8")
        courseName = courseName.replace(" ", "")
        courseName = courseName[0:len(courseName) - 3]
        expectedGPA = convertHtmlToString(rowList[8])
        receivedGPA = convertHtmlToString(rowList[9])
        if expectedGPA != 'N/A':
            expectedGPA = rowList[8].text.strip().encode("utf-8")[len(expectedGPA) - 5:len(expectedGPA) - 1]
        if receivedGPA != 'N/A':
            receivedGPA = rowList[9].text.strip().encode("utf-8")[len(receivedGPA) - 5:len(receivedGPA) - 1]

        if receivedGPA == 'N/A':
            continue
    
        if courseName in dirtyUnpairedCourseNameHash:
            if (convertHtmlToString(rowList[2])[0:2] == 'S1' or convertHtmlToString(rowList[2])[0:2] == 'S2' or convertHtmlToString(rowList[2])[0:2] == 'S3' or convertHtmlToString(rowList[2])[0:2] == 'SU'):       
                scrapeCleanedUnpairedRows.append((courseName, 'S', convertHtmlToString(rowList[2]), convertHtmlToString(rowList[3]), convertHtmlToString(rowList[4]), convertHtmlToString(rowList[5]), convertHtmlToString(rowList[6]), convertHtmlToString(rowList[7]), expectedGPA, receivedGPA))  # Get rid of empty values\
            else:
                scrapeCleanedUnpairedRows.append((courseName, 'NS', convertHtmlToString(rowList[2]), convertHtmlToString(rowList[3]), convertHtmlToString(rowList[4]), convertHtmlToString(rowList[5]), convertHtmlToString(rowList[6]), convertHtmlToString(rowList[7]), expectedGPA, receivedGPA))  # Get rid of empty values\



def initializeDefaultDict():
    for i in scrapeCleanedUnpairedRows:
        UnpairedRowsWithSeparateChainingDict[i[0]].append(i)


def pairDefaultDict():
    for course in UnpairedRowsWithSeparateChainingDict.keys():
        summerCount = 0
        nonsummerCount = 0
        for tupe in UnpairedRowsWithSeparateChainingDict[course]:
            if tupe[1] == 'S':
                summerCount += 1
            else:
                nonsummerCount += 1
                
        if summerCount == 0:
            del UnpairedRowsWithSeparateChainingDict[course]
        
        if nonsummerCount == 0:
            del UnpairedRowsWithSeparateChainingDict[course]

            
def getListFromDictionary():
    for course in UnpairedRowsWithSeparateChainingDict.keys():
        for tupe in UnpairedRowsWithSeparateChainingDict[course]:
            cleanedDepartmentalCourseList.append(tupe)
    cleanedDepartmentalCourseList.sort()


def returnTupleListToStringList(tupleList):
    stringList = []
    for tupe in tupleList:
        stringList.append("%s,%s,%s,%s,%s,%s,%s,%s,%s,%s" % (tupe[0], tupe[1], tupe[2], tupe[3], tupe[4], tupe[5], tupe[6], tupe[7], tupe[8], tupe[9]))
    return stringList


def prettyPrintList(listOfStrings):
    for i in listOfStrings:
        print i

        
def resetAllLists():
    global dirtyDupedUnpairedCourseNameList
    global dirtyUnpairedCourseNameSet
    global dirtyUnpairedCourseNameHash
    global scrapeCleanedUnpairedRows
    global UnpairedRowsWithSeparateChainingDict
    global cleanedDepartmentalCourseList
    dirtyDupedUnpairedCourseNameList = []
    dirtyUnpairedCourseNameSet = []
    dirtyUnpairedCourseNameHash = {}  # boolean hash
    scrapeCleanedUnpairedRows = []
    UnpairedRowsWithSeparateChainingDict = defaultdict(list)
    cleanedDepartmentalCourseList = []


def scrapeAndInitializeList(link):
    '''Takes in a url for a department and returns a list of one-line strings for the department'''
    setLink(link)            
    scrapeNameList(dirtyDupedUnpairedCourseNameList)
    dirtyUnpairedCourseNameSet = createSetList(dirtyDupedUnpairedCourseNameList)
    booleanHashCourseName(dirtyUnpairedCourseNameSet)
    scrapeCapeRows()  # initializes scrapeCleanedUnpairedRows
    initializeDefaultDict()
    pairDefaultDict()
    getListFromDictionary()
    if link.lower() == 'https://cape.ucsd.edu/responses/Results.aspx?Name=&CourseNumber=se'.lower():
        SElist = [x for x in cleanedDepartmentalCourseList if x[0][0:2] == 'SE']
        return returnTupleListToStringList(SElist)
        return SElist
    return returnTupleListToStringList(cleanedDepartmentalCourseList)


# IMPORTANT: Each list is exactly named after a department and contains string elements that quote the entire row
#format: courseName, term, enroll#, evals made, recommend class%, recommend prof%, hrs/wk, GPA expected, GPA received 
#sry i kept row as an entire string and didn't break each row up into different data-type, leaving it to you 
#instead of individual departments, we could analyze 3 area studies (engineering, humanities, socialSciences)
#each area study list is a concatenation of the 5 most popular departments within that area study

CSE = scrapeAndInitializeList('https://cape.ucsd.edu/responses/Results.aspx?Name=&CourseNumber=cse')
ECE = scrapeAndInitializeList('https://cape.ucsd.edu/responses/Results.aspx?Name=&CourseNumber=ece')
MAE = scrapeAndInitializeList('https://cape.ucsd.edu/responses/Results.aspx?Name=&CourseNumber=mae')
SE = scrapeAndInitializeList('https://cape.ucsd.edu/responses/Results.aspx?Name=&CourseNumber=se')
NANO = scrapeAndInitializeList('https://cape.ucsd.edu/responses/Results.aspx?Name=&CourseNumber=nano')
engineering = CSE + ECE + MAE + SE + NANO
 
PHIL = scrapeAndInitializeList('https://cape.ucsd.edu/responses/Results.aspx?Name=&CourseNumber=phil')
HIEA = scrapeAndInitializeList('https://cape.ucsd.edu/responses/Results.aspx?Name=&CourseNumber=hiea')
MUS = scrapeAndInitializeList('https://cape.ucsd.edu/responses/Results.aspx?Name=&CourseNumber=mus')
TD = scrapeAndInitializeList('https://cape.ucsd.edu/responses/Results.aspx?Name=&CourseNumber=td')
VIS = scrapeAndInitializeList('https://cape.ucsd.edu/responses/Results.aspx?Name=&CourseNumber=vis')
humanities = PHIL + HIEA + MUS + TD + VIS
  
ECON = scrapeAndInitializeList('https://cape.ucsd.edu/responses/Results.aspx?Name=&CourseNumber=econ')
POLI = scrapeAndInitializeList('https://cape.ucsd.edu/responses/Results.aspx?Name=&CourseNumber=poli')
COGS = scrapeAndInitializeList('https://cape.ucsd.edu/responses/Results.aspx?Name=&CourseNumber=cogs')
PSYC = scrapeAndInitializeList('https://cape.ucsd.edu/responses/Results.aspx?Name=&CourseNumber=psyc')
MGT = scrapeAndInitializeList('https://cape.ucsd.edu/responses/Results.aspx?Name=&CourseNumber=mgt')
socialSciences = ECON + POLI + COGS + PSYC + MGT

In [14]:
#print engineering and create txt file
engineeringFile = open('engineering.txt', 'wb')
for item in engineering:
    engineeringFile.write("%s\n" % item)
#prettyPrintList(engineering)

In [15]:
#print humanitiesFile and create txt file
humanitiesFile = open('humanities.txt', 'wb')
for item in humanities:
    humanitiesFile.write("%s\n" % item)
#prettyPrintList(humanities)

In [16]:
#print socialSciences and create txt file
socialSciencesFile = open('socialSciences.txt', 'wb')
for item in socialSciences:
    socialSciencesFile.write("%s\n" % item)
#prettyPrintList(socialSciences)

In [17]:
#display
prettyPrintList(CSE)

CSE100-AdvancedDataStructures,NS,FA07,41,27,88.9,92.3,13.46,3.13,2.97
CSE100-AdvancedDataStructures,NS,FA08,58,35,82.9,85.7,14.56,3.18,2.72
CSE100-AdvancedDataStructures,NS,FA09,55,32,93.8,87.5,12.06,3.13,2.99
CSE100-AdvancedDataStructures,NS,FA10,139,75,93.3,80.0,10.82,3.23,2.95
CSE100-AdvancedDataStructures,NS,FA11,140,65,90.8,86.2,11.45,3.26,2.95
CSE100-AdvancedDataStructures,NS,FA12,236,96,90.1,86.4,10.33,3.38,3.13
CSE100-AdvancedDataStructures,NS,FA13,205,161,98.1,93.5,9.87,3.34,2.64
CSE100-AdvancedDataStructures,NS,FA14,183,131,92.9,89.8,10.75,3.13,3.02
CSE100-AdvancedDataStructures,NS,FA14,197,138,94.7,93.1,11.75,3.26,3.02
CSE100-AdvancedDataStructures,NS,FA15,148,54,94.1,92.2,9.60,3.31,3.14
CSE100-AdvancedDataStructures,NS,FA15,148,69,92.2,84.4,10.06,3.11,3.14
CSE100-AdvancedDataStructures,NS,FA15,79,42,90.2,65.9,9.00,3.50,3.17
CSE100-AdvancedDataStructures,NS,FA16,128,117,98.3,96.6,11.08,3.48,3.19
CSE100-AdvancedDataStructures,NS,FA16,137,117,88.8,71.6,10.95,3.31,2.55
CSE100-A

In [18]:
#display
prettyPrintList(HIEA)

HIEA126-TheSilkRoad,NS,FA13,51,27,96.2,96.2,5.81,3.39,3.46
HIEA126-TheSilkRoad,NS,SP09,56,40,97.5,100.0,4.20,3.61,3.30
HIEA126-TheSilkRoad,S,S112,30,5,100.0,100.0,7.30,3.50,3.32
HIEA126-TheSilkRoad,S,S117,24,15,93.3,92.9,7.03,3.46,3.36
HIEA126-TheSilkRoad,S,S214,21,2,100.0,100.0,7.50,3.50,2.93
HIEA128-History/MaterialCultureChina,NS,FA09,63,31,93.3,100.0,3.90,3.64,3.47
HIEA128-History/MaterialCultureChina,NS,SP14,31,10,66.7,66.7,6.50,3.43,2.61
HIEA128-History/MaterialCultureChina,NS,WI09,62,41,95.1,97.5,2.85,3.61,3.17
HIEA128-History/MaterialCultureChina,NS,WI12,63,25,92.0,100.0,4.82,3.54,3.30
HIEA128-History/MaterialCultureChina,S,S212,20,4,100.0,100.0,8.00,3.67,3.29
HIEA128-History/MaterialCultureChina,S,S214,20,4,100.0,100.0,5.00,3.25,3.20
HIEA131-China/War&Revolution1911-49,NS,FA07,51,47,97.9,97.9,5.15,3.55,3.15
HIEA131-China/War&Revolution1911-49,NS,FA08,106,52,92.0,96.2,5.19,3.45,3.02
HIEA131-China/War&Revolution1911-49,NS,FA09,76,61,93.3,95.0,5.42,3.50,3.35
HIEA131-China/War&Rev

In [19]:
#display
prettyPrintList(MGT)

MGT103-ProductMarketing&Management,NS,FA08,55,24,95.7,100.0,4.50,3.57,3.39
MGT103-ProductMarketing&Management,NS,FA08,62,45,95.5,95.3,3.95,3.55,3.39
MGT103-ProductMarketing&Management,NS,FA09,59,29,100.0,100.0,2.90,3.58,3.22
MGT103-ProductMarketing&Management,NS,FA09,65,44,92.9,90.5,3.91,3.66,3.22
MGT103-ProductMarketing&Management,NS,FA10,31,13,92.3,84.6,5.88,3.85,3.47
MGT103-ProductMarketing&Management,NS,FA10,68,30,96.6,89.3,3.23,3.81,3.47
MGT103-ProductMarketing&Management,NS,FA11,80,60,91.5,86.4,4.43,3.67,3.39
MGT103-ProductMarketing&Management,NS,FA11,80,64,93.8,89.1,4.04,3.64,3.39
MGT103-ProductMarketing&Management,NS,FA12,74,62,100.0,100.0,4.50,3.62,3.47
MGT103-ProductMarketing&Management,NS,FA12,76,59,94.9,94.9,5.60,3.83,3.36
MGT103-ProductMarketing&Management,NS,FA12,78,63,93.7,84.1,3.79,3.66,3.36
MGT103-ProductMarketing&Management,NS,FA12,79,60,93.1,87.9,3.76,3.70,3.36
MGT103-ProductMarketing&Management,NS,FA13,76,69,91.3,94.2,4.36,3.54,3.10
MGT103-ProductMarketing&Managemen

MGT5-ManagerialAccounting,NS,WI17,109,107,95.3,97.2,4.71,3.55,3.57
MGT5-ManagerialAccounting,NS,WI17,165,144,93.8,93.1,4.64,3.44,3.21
MGT5-ManagerialAccounting,NS,WI18,125,117,95.7,97.4,3.20,3.48,3.53
MGT5-ManagerialAccounting,NS,WI18,157,114,91.0,93.6,4.14,3.43,2.94
MGT5-ManagerialAccounting,S,S111,74,21,90.5,76.2,7.07,3.60,2.78
MGT5-ManagerialAccounting,S,S112,68,16,93.8,93.8,5.00,3.33,2.99
MGT5-ManagerialAccounting,S,S113,37,6,100.0,100.0,5.17,3.83,2.87
MGT5-ManagerialAccounting,S,S114,35,6,83.3,83.3,6.90,3.40,3.03
MGT5-ManagerialAccounting,S,S115,29,7,100.0,100.0,6.17,3.43,3.26
MGT5-ManagerialAccounting,S,S116,48,14,100.0,85.7,5.27,3.43,3.18
MGT5-ManagerialAccounting,S,S117,55,22,95.2,95.2,5.36,3.50,2.76
